In [1]:
!pip install torch torchvision

In [2]:
!pip install clip-by-openai

  Using cached clip_by_openai-1.1-py3-none-any.whl (1.4 MB)
  Using cached ftfy-6.2.0-py3-none-any.whl (54 kB)
INFO: pip is looking at multiple versions of clip-by-openai to determine which version is compatible with other requirements. This could take a while.
  Using cached clip_by_openai-1.0.1-py3-none-any.whl (1.4 MB)
  Using cached clip_by_openai-0.1.1.5-py3-none-any.whl (1.4 MB)
  Using cached clip_by_openai-0.1.1.4-py3-none-any.whl (1.4 MB)
  Using cached clip_by_openai-0.1.1.3-py3-none-any.whl (1.4 MB)
  Using cached clip_by_openai-0.1.1.2-py3-none-any.whl (1.4 MB)
  Using cached clip_by_openai-0.1.1-py3-none-any.whl (13 kB)
  Using cached clip_by_openai-0.1.0-py3-none-any.whl (13 kB)
INFO: pip is looking at multiple versions of clip-by-openai to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install clip-by-openai==0.1.0, clip-by-openai==0.1.1, clip-by-openai==0.1.1.2, clip-by-openai==0.1.1.3, clip-by-openai==0.1.1.4, clip

In [4]:
!git clone https://github.com/openai/CLIP.git

Cloning into 'CLIP'...
remote: Enumerating objects: 251, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 251 (delta 3), reused 3 (delta 0), pack-reused 243
Receiving objects: 100% (251/251), 8.93 MiB | 12.42 MiB/s, done.
Resolving deltas: 100% (127/127), done.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [5]:
!pip install ftfy regex tqdm

  Using cached ftfy-6.2.0-py3-none-any.whl (54 kB)


In [6]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-ioyj9pmp
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-ioyj9pmp
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369499 sha256=c3a0462decdbbdc819b6d91b77dd3b3df58772105bea019cdd07e151c72e06ff
  Stored in directory: /tmp/pip-ephem-wheel-cache-q8iqvseu/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [29]:
import torch
import clip
from PIL import Image
import os

# CLIP 모델 로드
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# 이미지 파일들을 로드하고 전처리
def load_images(image_folder):
    images = []
    image_files = [os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith(('png', 'jpg', 'jpeg'))]

    for image_file in image_files:
        image = preprocess(Image.open(image_file)).unsqueeze(0).to(device)
        images.append((image_file, image))

    return images

# 이미지 임베딩 계산
def get_image_features(images):
    image_features = []
    for image_file, image in images:
        with torch.no_grad():
            image_feature = model.encode_image(image)
            image_features.append((image_file, image_feature))
    return image_features

# 텍스트 쿼리 임베딩 계산
def get_text_features(text_query):
    with torch.no_grad():
        text_tokens = clip.tokenize([text_query]).to(device)
        text_features = model.encode_text(text_tokens)
    return text_features

# 유사도 계산
def calculate_similarity(image_features, text_features):
    similarities = []
    for image_file, image_feature in image_features:
        similarity = torch.cosine_similarity(image_feature, text_features, dim=1).item()
        similarities.append((image_file, similarity))
    return similarities

# 이미지 검색 함수
def search_images(image_folder, text_query):
    global image_features
    images = load_images(image_folder)
    image_features = get_image_features(images)
    text_features = get_text_features(text_query)

    similarities = calculate_similarity(image_features, text_features)
    similarities.sort(key=lambda x: x[1], reverse=True)

    return similarities

# 이미지 폴더와 텍스트 쿼리 설정
image_folder = '/content/image'
text_query = "강아지 캐릭터"

# 이미지 검색
results = search_images(image_folder, text_query)

# 결과 출력
for image_file, similarity in results:
    print(f"Image: {image_file}, Similarity: {similarity}")


Image: /content/image/깜자_목도리.jpg, Similarity: 0.28266990184783936
Image: /content/image/화면 캡처 2023-02-24 154706.png, Similarity: 0.2757948040962219
Image: /content/image/노말틱 방송출현 ㅋㅋ.png, Similarity: 0.2611384391784668
Image: /content/image/ㄷㄷ.png, Similarity: 0.26030877232551575
Image: /content/image/KakaoTalk_20220930_024230736.jpg, Similarity: 0.25212395191192627
Image: /content/image/깜자_프로그래머.jpg, Similarity: 0.23917140066623688
Image: /content/image/Kitty_Gun.jpg, Similarity: 0.2333204746246338
Image: /content/image/파이썬 3000분.jpg, Similarity: 0.21473349630832672
Image: /content/image/hacker.png, Similarity: 0.2023203819990158
Image: /content/image/KakaoTalk_20230417_212253636.jpg, Similarity: 0.20035888254642487
Image: /content/image/KakaoTalk_20231112_194428701_01.jpg, Similarity: 0.19665507972240448
Image: /content/image/Screenshot 2023-11-12 at 6.58.04 PM.png, Similarity: 0.17402087152004242
Image: /content/image/Screenshot from 2023-11-12 19-06-14.png, Similarity: 0.17266601324

In [37]:
import torch
import clip
from PIL import Image
import os

# CLIP 모델 로드
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# 이미지 파일들을 로드하고 전처리
def load_images(image_folder):
    images = []
    image_files = [os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith(('png', 'jpg', 'jpeg'))]

    for image_file in image_files:
        image = preprocess(Image.open(image_file)).unsqueeze(0).to(device)
        images.append((image_file, image))

    return images

# 이미지 임베딩 계산
def get_image_features(images):
    image_features = []
    for image_file, image in images:
        with torch.no_grad():
            image_feature = model.encode_image(image)
            image_features.append((image_file, image_feature))
    return image_features

# 해시태그 임베딩 계산
def get_hashtag_features(hashtags):
    with torch.no_grad():
        text_tokens = clip.tokenize(hashtags).to(device)
        text_features = model.encode_text(text_tokens)
    return text_features

# 유사도 계산
def calculate_similarity(image_features, text_features):
    similarities = []
    for image_file, image_feature in image_features:
        similarity = torch.cosine_similarity(image_feature, text_features, dim=1)
        similarities.append((image_file, similarity))
    return similarities

# 해시태그 생성 함수
def generate_hashtags(image_folder, hashtags):
    images = load_images(image_folder)
    image_features = get_image_features(images)
    hashtag_features = get_hashtag_features(hashtags)

    similarities = calculate_similarity(image_features, hashtag_features)

    image_hashtags = {}
    for image_file, similarity in similarities:
        # 가장 유사한 해시태그 5개 선택
        top_indices = similarity.topk(3).indices.cpu().numpy()
        image_hashtags[image_file] = [hashtags[i] for i in top_indices]

    return image_hashtags

# 이미지 폴더와 해시태그 리스트 설정
image_folder = './'  # 여기에 이미지 폴더 경로를 입력하세요
# hashtags = [
#     "#nature", "#travel", "#food", "#art", "#technology",
#     "#fashion", "#sports", "#music", "#animals", "#people"
# ]
hashtags = [
    "#character", "#people", "#travel", "#technology", "#art",
    "#animals", "#computer", "#fashion", "#game", "#mobile"
]
# hashtags = [
#     "#캐릭터","#사람","#여행","#기술","#예술","#동물","#컴퓨터","#패션", "#게임", "#모바일"
# ]

# 해시태그 생성
image_hashtags = generate_hashtags(image_folder, hashtags)

# 결과 출력
for image_file, tags in image_hashtags.items():
    print(f"Image: {image_file}, Hashtags: {tags}")


Image: ./Screenshot from 2023-11-12 19-07-11.png, Hashtags: ['#computer', '#technology', '#people']
Image: ./KakaoTalk_20220930_024230736.jpg, Hashtags: ['#mobile', '#technology', '#game']
Image: ./파이썬 3000분.jpg, Hashtags: ['#computer', '#people', '#mobile']
Image: ./Screenshot from 2023-11-12 19-06-14.png, Hashtags: ['#computer', '#technology', '#people']
Image: ./Screenshot 2023-11-12 at 6.58.04 PM.png, Hashtags: ['#computer', '#technology', '#people']
Image: ./Kitty_Gun.jpg, Hashtags: ['#game', '#art', '#animals']
Image: ./Screenshot from 2023-11-12 19-06-42.png, Hashtags: ['#computer', '#technology', '#people']
Image: ./노말틱 방송출현 ㅋㅋ.png, Hashtags: ['#computer', '#game', '#people']
Image: ./Microsoft Weekday.png, Hashtags: ['#technology', '#people', '#computer']
Image: ./Screenshot from 2023-11-12 19-07-20.png, Hashtags: ['#computer', '#technology', '#people']
Image: ./23.06.14 backup.png, Hashtags: ['#computer', '#game', '#technology']
Image: ./깜자_목도리.jpg, Hashtags: ['#travel', '#ar

In [24]:
!pwd

/content/image


In [12]:
image_features

[('/content/image/Screenshot from 2023-11-12 19-07-11.png',
  tensor([[ 9.6791e-02,  3.7852e-02, -1.9908e-01, -1.3949e-01,  3.1507e-01,
           -2.3714e-01,  3.0505e-01, -4.9289e-01, -1.6138e-01, -2.2989e-01,
            2.7992e-01, -3.0821e-01, -8.7734e-01, -4.3039e-02,  5.4054e-03,
           -2.0790e-01,  3.7204e-01,  1.6885e-01, -1.5865e-01,  1.9372e-01,
            1.3082e-01, -2.0191e-01, -4.9769e-02, -2.9671e-02, -2.8709e-01,
            2.1456e-03, -3.1537e-01,  3.3819e-01,  1.3385e-02,  1.6927e-01,
            1.0746e-01,  1.8744e-01, -1.8724e-02,  5.1781e-02,  9.7931e-02,
           -2.3115e-01, -2.6282e-02,  3.2438e-01, -2.3995e-02, -1.9226e+00,
           -2.6954e-01, -5.6339e-02, -3.3615e-01, -6.0446e-01,  1.2659e-01,
            7.6365e-02,  1.4468e-01,  1.0914e-01, -3.1390e-01,  3.2306e-01,
           -4.9528e-02, -1.2103e-01, -1.2808e-02, -2.8427e-01, -1.8452e-01,
            2.7531e-01,  1.7428e-01, -2.7206e-02,  5.8775e-02,  2.2027e-01,
           -8.9125e-01,  4.7

In [22]:
image_features[0][1].shape

torch.Size([1, 512])

In [17]:
ls -1 | wc -l

23
